## Endpoint execution

This Jupyter Notebook simplifies interaction with the key endpoints of the Capture The Flag platform, offering a user-friendly environment to test and utilize its functionalities. It bridges the gap between backend mechanics and practical applications, making it a valuable resource for developers and enthusiasts.

### Initial imports and configurations

In [80]:
import shutil
import requests
import urllib3

s = requests.session()

# Needed locally only
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
s.verify = False

# Mandatory configuration
keycloak_host = "https://myhost/keycloak"
deployer_host = "https://deployer.local.lan"

# Paths to the challenge files
src_folder_location = "../challenge/src/"
solution_folder_location = "../challenge/solution/"
challenge_yaml_location = "../challenge/challenge.yml"

# Login credentials
username = "kilar0276"
password = "Test1234"

### Get access token
Access token is valid for 5 minutes

In [81]:
r = s.post(
    f"{keycloak_host}/realms/ctf/protocol/openid-connect/token/", 
    data={
        "client_id":"deployer", 
        "username": username, 
        "password": password, 
        "grant_type": "password", 
        "scope": "openid"
    }, 
    timeout=20
)
print("login:", r.status_code, r.content)
r.raise_for_status()

s.headers = {"Authorization": "Bearer " + r.json().get("access_token")}

login: 200 b'{"access_token":"eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJSTFhFZnpRbE9YWUZkQUxJb1V4YzRFNS0wUDBLdnphQnhSLTloSzl2a2xJIn0.eyJleHAiOjE3NDMwOTkwMTEsImlhdCI6MTc0MzA5ODcxMSwianRpIjoiZDg3MmI5YTctNTg5Mi00ZDhmLTg0YjktZDBiNDNiOTIyNjhhIiwiaXNzIjoiaHR0cHM6Ly9teWhvc3Qva2V5Y2xvYWsvcmVhbG1zL2N0ZiIsImF1ZCI6WyJjdGZkIiwic3RlcCIsImFjY291bnQiXSwic3ViIjoiN2UyOWY2Y2UtMTAwMS00ZTFkLWFjNzItNGI4ZjRiMmFlZjNmIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiZGVwbG95ZXIiLCJzaWQiOiI4YTFkNTQyMi01ZTM4LTRhYjMtYWFhYy05NDE1OWMxYzk2MDciLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbIiJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1jdGYiLCJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiY3RmZCI6eyJyb2xlcyI6WyJhZG1pbiJdfSwic3RlcCI6eyJyb2xlcyI6WyJiYXN0aW9uIl19LCJkZXBsb3llciI6eyJyb2xlcyI6WyJhZG1pbiJdfSwiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwibmFtZSI6IktpYW

### Post challenge to deployer service

In [82]:
shutil.make_archive("challenge", "zip", src_folder_location)
shutil.make_archive("solution", "zip", solution_folder_location)

r = s.post(
    deployer_host + "/challenges", 
    files=[
        ("upload[]", open("challenge.zip", "rb")),
        ("upload[]", open(challenge_yaml_location, "rb")),
        ("upload[]", open("solution.zip", "rb"))
    ], 
    timeout=20
)
print("add challenge:", r.status_code, r.content)
r.raise_for_status()

challenge_id = r.json().get("challengeid")

add challenge: 200 b'{"challengeid":"5ec86582-9cf4-4453-b642-2c72484e41aa"}'


### Submit challenge to CTFd

In [83]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/publish", timeout=20)
print("publish:", r.status_code, r.content)

publish: 201 b'{}'


### Start the Challenge using the API

In [84]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/start", timeout=20)
print("start challenge:", r.status_code, r.content)
r.raise_for_status()

start challenge: 200 b'{"url":"dbbaa940-158f-4a36.deployer.local.lan","secondslseft":7200,"started":true,"verified":false}'


### Start Test using the API

In [85]:
r = s.post(deployer_host + f"/solutions/{challenge_id}/start", timeout=20)
print("start test:", r.status_code, r.content)
r.raise_for_status()

start test: 200 b'{"url":"dbbaa940-158f-4a36.deployer.local.lan","secondslseft":7200,"started":true,"verified":false}'


### Verify flag for a Challenge

In [ ]:
data = {
    "flag": "flag{ssh-example}"
}

r = s.post(deployer_host + f"/challenges/{challenge_id}/verify", timeout=20, json=data)
print("Test flag:", r.status_code, r.content)
r.raise_for_status()

### Stop the Challenge using the API

In [ ]:
r = s.post(deployer_host + "/challenges/" + challenge_id + "/stop", timeout=20)
print("stop challenge:", r.status_code, r.content)
r.raise_for_status()

### Stop the Test using the API

In [ ]:
r = s.post(deployer_host + "/solutions/" + challenge_id + "/stop", timeout=20)
print("stop test:", r.status_code, r.content)
r.raise_for_status()

### Get Logs (not very beneficial)

In [ ]:
r = s.get(deployer_host + "/challenges/" + challenge_id + "/logs", timeout=20)
print("logs:", r.status_code, r.content)
r.raise_for_status()

### SSH command

The following SSH command demonstrates how to connect to a CTF Challenge using SSH, leveraging the Bastion as a jump proxy while enabling port forwarding from the container to your local machine. This method allows access to a wider range of ports beyond those directly exposed by the platform itself.

```bash
ssh <username>@ssh.<subdomain> -p 8022 -J bastion@ctf.jacopomauro.com:443 -L <local-port>:<service:port>
sshpass -p 'password' ssh -o StrictHostKeyChecking=accept-new -p 8022 <user>@ssh.challenge-<subdomain> -J bastion@ctf.jacopomauro.com:443 -L <local-port>:<service:port> -N &
```